In [1]:
# Imports
from sys import argv
from os import path
from enum import Enum
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
# Constants

NUMEXEC = 10
NUMFOLD = 10
SEEDFACTOR = 10
DATASET = '../dataset/'
RESULT = '../result/'
FRAMES = [4,5,6,7,8]
DATASETTYPE = 'va'
RESULTFILE = 'results.csv'
LEARNINGRATE = 0.01
BATCHSIZE = 10
HIDDENSIZE = 10
TESTFRAC = 0.1

PARAM = "-L 0.3 -M 0.2 -N 500 -V 0 -S %%SEED%% -E 20 -H %%HIDDEN_LAYERS%%"
HIDDENLAYER = ["a", "40", "30", "20", "10"]

class CoordinateEnum(Enum):
    X = 'X'
    Y = 'Y'
    Z = 'Z'

class Sensor:
    def __init__(self, name, key, *argv):
        self.name = name
        self.key = key
        self.coordinates = list(argv)   

class SensorEnum(Enum):
	LINEAR_ACCELERATION_EARTH = Sensor("aceleracaoLinearTerra", "AclLinE", CoordinateEnum.X, CoordinateEnum.Y, CoordinateEnum.Z)
	ACCELEROMETER_EARTH = Sensor("acelerometroTerra", "AcelE", CoordinateEnum.X, CoordinateEnum.Y, CoordinateEnum.Z)
	MAGNETIC_FIELD_EARTH = Sensor("campoMagneticoTerra", "MagE", CoordinateEnum.Y, CoordinateEnum.Z)
	GYROSCOPE_EARTH = Sensor("giroscopioTerra", "GirE", CoordinateEnum.X, CoordinateEnum.Y, CoordinateEnum.Z)

class EventEnum(Enum):
    AGGRESSIVE_LEFT_TURN = "curva_esquerda_agressiva"
    AGGRESSIVE_RIGHT_TURN = "curva_direita_agressiva"
    AGGRESSIVE_LEFT_LANE_CHANGE = "troca_faixa_esquerda_agressiva"
    AGGRESSIVE_RIGHT_LANE_CHANGE = "troca_faixa_direita_agressiva"
    AGGRESSIVE_ACCELERATION = "aceleracao_agressiva"
    AGGRESSIVE_BRAKE = "freada_agressiva"
    NON_AGGRESSIVE = "evento_nao_agressivo"
 
events = {'curva_esquerda_agressiva': 0, 
          'curva_direita_agressiva': 1, 
          'troca_faixa_esquerda_agressiva': 2, 
          'troca_faixa_direita_agressiva': 3, 
          'aceleracao_agressiva': 4, 
          'freada_agressiva': 5, 
          'evento_nao_agressivo': 6
         }

In [3]:
class ClassifierExecuter:
    def __init__(self,*argv) -> None:
        if(len(argv) > 1):
            sensor,coordinate,hiddenLayers,numframes,seeds = argv
            self.sensor = sensor
            self.coordinate = coordinate
            self.Param = PARAM.replace("%%HIDDEN_LAYERS%%",hiddenLayers)
            self.numframes = numframes
            self.seeds = seeds
            self.isDir = True
        else:
            filename = argv[0]
            self.file = filename
            self.isDir = False

In [4]:
class RecordHandler:
    def __init__(self,values,events):
        self.values = values
        self.events = events
        self.numClasses = len(events)
        self.inputSize = values[0][:-2].size
    
    def segregate(self, testFrac):
        
        np.random.shuffle(self.values)
    
        features = torch.Tensor(
            np.array( [i[:-2].astype(np.float32) for i in self.values])
        )
        
        labels = torch.Tensor(
            np.array( [ self.events[i[-1]] for i in self.values])
        ).type(torch.LongTensor)
        
        #TODO: Also store quadro0 for future refrences (giving results)
        
        testSize = int(testFrac*len(features))
        
        self.testFeatures = features[:testSize]
        self.testLabels = labels[:testSize]
        
        self.trainFeatures = features[testSize:]
        self.trainLabels = labels[testSize:]


In [5]:
class NeuralNet(nn.Module):
    def __init__(self, inputSize, hiddenSize, numClasses) -> None:
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(inputSize,hiddenSize)
        self.l2 = nn.Linear(hiddenSize,hiddenSize)
        self.l3 = nn.Linear(hiddenSize,hiddenSize)
        self.l4 = nn.Linear(hiddenSize,numClasses)

    def forward(self,x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = self.l4(x)
        return F.log_softmax(x,dim=1)


In [6]:
def train():
    # train
    for epoch in range(NUMEXEC):
        batch_loss = 0
        for i in tqdm(range(0, len(handler.trainFeatures), BATCHSIZE)): 

            features = handler.trainFeatures[i:i+BATCHSIZE].view(-1,handler.inputSize)
            labels = handler.trainLabels[i:i+BATCHSIZE]

            net.zero_grad()

            # forward
            outputs = net(features)
#             print(outputs)
            loss = F.nll_loss(outputs,labels)
#             print(outputs)
#             print(labels)
#             print()
#             loss = nn.CrossEntropyLoss()
#             loss = loss(outputs,labels)

            # backward
            loss.backward()    
            optimizer.step()
            batch_loss = batch_loss + loss
        print(batch_loss)
    


In [7]:
def predictTest():
    # test
    with torch.no_grad():
        correct = 0
        samples = 0

        for i in range(len(handler.testFeatures)):
            features = handler.testFeatures[i].view(-1,handler.inputSize)
            label = handler.testLabels[i]

            output = net(features)
            if(torch.argmax(output) == label):
                correct += 1
            samples += 1 

        acc = 100.0 * correct /samples
        print(f'accuracy = {acc}')


In [8]:
def predictTrain():
    # test
    with torch.no_grad():
        correct = 0
        samples = 0

        for i in range(len(handler.trainFeatures)):
            features = handler.trainFeatures[i].view(-1,handler.inputSize)
            label = handler.trainLabels[i]

            output = net(features)
#             print(torch.argmax(output))
            if(torch.argmax(output) == label):
                correct += 1
            samples += 1 

        acc = 100.0 * correct /samples
        print(f'accuracy = {acc}')

In [9]:
#         TESTING
# for i in FRAMES:
#     for j in SensorEnum:
#         path = f'../datasets/va_{j.value.name}_nq{i}.csv'
#         file = pd.read_csv(path)
#         values = file.values
#         handler = RecordHandler(values,events)
#         handler.segregate(TESTFRAC)
#         net = NeuralNet(handler.inputSize, HIDDENSIZE, handler.numClasses)
#         optimizer = optim.Adam(net.parameters(),lr = LEARNINGRATE) 
#         train()
#         predictTrain()
#         predictTest()
#         print(i,j)

        
path = f'../datasets/va_acelerometroTerra_nq8.csv'
file = pd.read_csv(path)
values = file.values
handler = RecordHandler(values,events)
handler.segregate(TESTFRAC)
net = NeuralNet(handler.inputSize, HIDDENSIZE, handler.numClasses)
optimizer = optim.Adam(net.parameters(),lr = LEARNINGRATE) 

In [10]:
train()
predictTrain()
predictTest()


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 468.26it/s]


tensor(91.1998, grad_fn=<AddBackward0>)


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 467.68it/s]


tensor(75.7058, grad_fn=<AddBackward0>)


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 415.38it/s]


...

tensor(39.8969, grad_fn=<AddBackward0>)


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 413.31it/s]


tensor(37.0677, grad_fn=<AddBackward0>)


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 486.96it/s]


tensor(35.2787, grad_fn=<AddBackward0>)
accuracy = 69.97971602434077
accuracy = 68.51851851851852
